In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
nb_path = '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
!pip install Bio

In [ ]:
import os
os.chdir("/content/drive/MyDrive")

import scipy
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import glob

os.chdir('/content/drive/MyDrive/Colab Notebooks')
import matplotlib.pyplot as plt
import numpy as np
# from fuzzywuzzy import process
import gzip
from rapidfuzz import process

# Installation things (run if you are a new user)

In [ ]:
import os
import sys

nb_path = '/content/drive/MyDrive'
if nb_path not in sys.path:
  sys.path.insert(5, nb_path)
# os.symlink('/content/notebooks', nb_path)
os.chdir("/content/drive/MyDrive")

#pip install --target=$nb_path abifpy #change package name here

# Import files

In [ ]:
import glob
# rewrite all files in the dir as .fastq, change read1/2 as needed

dir = '/content/drive/MyDrive/Colab Notebooks/FolderName'
# save to fastq
files = glob.glob((dir + "/*READ1-Sequences.txt.gz"))
for f in files:
  s = str(os.path.basename(f))
  print(s)
  s = s.replace('.txt.gz','')
  with gzip.open(f, "rt") as handle:
    records = (rec for rec in SeqIO.parse(handle, "fastq"))
    count = SeqIO.write(records, str(s +".fastq"), "fastq")
    print(count)# rename and re-write all files as fastq
  handle.close()





# pair Read1 + Read2

Pairs read1 and read2

In [ ]:
#set working directory
dir = "/content/drive/MyDrive/Colab Notebooks/"


In [ ]:
# from Bio import SeqIO
# files = glob.glob(dir + "/*.fastq")
# for file in files:
# Get the lengths and ids, and sort on length
# base=os.path.basename(file)
# file_name = os.path.splitext(base)[0]
# file_name = os.path.splitext(file_name)[0]
# print(file_name)
with gzip.open(dir+file_name +"_merged1_fastq", "rt") as handle:
  records = (rec for rec in SeqIO.parse(handle, "fastq"))
  count = SeqIO.write(records, str(file_name+".fastq"), "fastq")
print(count)
handle.close()


merge fastq files

In [ ]:
file_name = 'mR417-L1-P4-ACAGTG'
barcode = 'ACAGTG'

In [ ]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import itertools

#Setup variables (could parse command line args instead)
file_f = file_name+"-READ1-Sequences.fastq"
file_r = file_name+"-READ2-Sequences.fastq"
file_out = file_name+"_merge.fastq"
format = "fastq" #or "fastq-illumina", or "fasta", or ...


def interleave(iter1, iter2) :
    for (forward, reverse) in zip(iter1,iter2):
        assert forward.id == reverse.id
        forward.id += "/1"
        reverse.id += "/2"
        yield forward
        yield reverse

records_f = SeqIO.parse(open(file_f,"r"), format)
records_r = SeqIO.parse(open(file_r,"r"), format)

handle = open(file_out, "w")
count = SeqIO.write(interleave(records_f, records_r), handle, format)
print(count)
handle.close()


PhredFilter only > 20


In [ ]:
# save to csv, only quality reads

df = pd.DataFrame(columns = ['Seq', 'Line#', "PhredScore"])
records=[]
for i, rec in enumerate(SeqIO.parse(file_name+"-READ1-Sequences.fastq", "fastq")):
  df.loc[len(df)] = [str(rec.seq), i+1, score]
  if min(rec.letter_annotations["phred_quality"]) >= 20:
    score = rec.letter_annotations["phred_quality"]
    records.append(rec)
    df.loc[len(df)] = [str(rec.seq), i+1, score]
    # print(df)
print(len(df))
df.to_csv("allseqs_"+barcode+".csv")
count = SeqIO.write(records, str(file_name+"_quality.fastq"), "fastq")
print(count)

In [ ]:
plt.hist(all_scores, bins = 100)
plt.xlabel ('Phred Score')
plt.ylabel('Count')

#Begin analysis of fastq files merged using PEAR

## Decoding sequence to position-based indexers



In [ ]:
file_name = "/content/drive/MyDrive/Colab Notebooks/NEW DEL19 FILES/20240221 DEL19 Cell Screen 1/mR417-L1-P1-ATGTCA"
barcode= "ATCACG"
mm= 0

##trim and save encoding regions by position index


In [ ]:
from Bio import Align
import io
import csv
from csv import writer
import rapidfuzz

seqs = pd.DataFrame()
decoding_table = pd.read_csv('6_Pos_Oligos_NGS_Fwd.csv')

df = pd.DataFrame(columns = ['Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'UMI','Line#'])
array = np.empty((len(seqs), 8))

output = io.StringIO()
writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
writer.writerow(np.array([1,2,3,4,5,6,7,8]))

for i, read in enumerate(SeqIO.parse(file_name+".fastq", "fastq")):


  ref='GCCGCCGCCTTCGTCCTTCTCAGCGACATGGxxxxxxxxTCAxxxxxxxxGTTxxxxxxxxCTAxxxxxxxxTTCxxxxxxxxCGCxxxxxxxxGCCTCCCAAACxxxxxxxxGTTTGCCCGCCAGTTGTTGTGCCAC'

  aligner = Align.PairwiseAligner()
  aligner.mismatch_score = -1
  aligner.internal_gap_score = -1
  alignments = aligner.align(ref, read)


  i1 = 28
  i2 = 39
  add_array = []
  while i2 < 95:
    o1 = str(read.seq[i1:i2])
    best = rapidfuzz.process.extract(o1, decoding_table['Sequences'], score_cutoff = 95, limit = 1)

    if best:#if match was found

      oligo_code = decoding_table.loc[decoding_table['Sequences'] == str(best[0][0])]['Tag'].item()

    else:
      oligo_code = ""
    add_array.append(str(oligo_code))
    i1 = i2
    i2 += 11

  add_array.append(read.seq[105:113])
  add_array.append(i)
  #print(add_array)
  #UMI
  if add_array[0][0:2] == "11" and add_array[1][0:2] == "22" and add_array[2][0:2] == "13" and add_array[3][0:2] == "24" and add_array[4][0:2] == "15" and add_array[5][0:2] == "26":
    writer.writerow(add_array)
  if i%100 == 0:
    print(i)

output.seek(0) # we need to get back to the start of the BytesIO
df = pd.read_csv(output, dtype = str)
df.columns = ['Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'UMI','Line#']
print(df)

df['Code'] = list(zip(df.Pos1, df.Pos2, df.Pos3,df.Pos4,df.Pos5, df.Pos6 ))
#print(df['new'])
df.to_csv('rapid_proc_%s_decodes_%iMM.csv' % (barcode, mm))

##Group "beads" and collect Unique UMI's for each bead

In [ ]:
barcode= "ATCACG"
mm= 0

In [ ]:
# collapse all reads and find unique UMI's for them.

df = pd.read_csv('rapid_proc_%s_decodes_%iMM.csv' % (barcode, mm))#"Code_matches_"+barcode+"_2MM.csv")
df = df.replace('nan', np.nan).dropna(axis=0)


UMIenriched_df = pd.DataFrame(df.groupby(['Code','UMI']).nunique()).reset_index()
print(UMIenriched_df)
# UMIenriched_df.columns = ['Code', 'UMI', 'UMICount', 'ReadCount']
print(UMIenriched_df)
UMIenriched_df.to_csv(barcode +"_perumicount.csv")#"collapsed_reads_perUMIcount_"+barcode+"_2MM.csv")

grouped_df = pd.DataFrame(df.groupby(['Code'], as_index = False).agg(UMI_unique=('UMI', 'unique'),UMI_count=('UMI', 'count'), Linecount=('Line#','count')).reset_index())

print(grouped_df)
grouped_df.to_csv(barcode +" groupedUMI.csv")#collapsed_reads_groupedUMI_"+barcode+"_2MM.csv")



counts_df = pd.DataFrame(df.groupby('Code')[['UMI','Line#']].nunique())
counts_df.columns = ['UniqueUMI','UniqueLine#']
counts_df.to_csv(barcode +"umicounts.csv")#collapsed_reads_UMIcounts_"+barcode+"_2MM.csv")

Conduct UMI string condensation to condense any UMIs which are only one nucleotide apart (HD = 1) and may be a result of mutation.

In [ ]:
df = pd.read_csv('rapid_proc_%s_decodes_%iMM.csv' % (barcode, mm))
# display(df)
test_df= df.groupby(['Code'], as_index = False).agg(UMI_unique=('UMI', 'unique'),UMI_count=('UMI', 'count'), Linecount=('Line#','count')).reset_index()

display(test_df)


In [ ]:
# test_df = pd.read_csv(barcode +" groupedUMI.csv")
import math
def hamming(s1, s2):
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

test_df['UMIs_condensed'] = ""
test_df['UMIs_condensed_count'] = 0

for idx,row in test_df.iterrows():
  result = []
  pieces = row['UMI_unique']

  if row['UMI_count'] >= 2:

    pieces = row['UMI_unique']
    # print(pieces)
    # print(len(pieces))
    thischunk=[]
    for piece in pieces:
      if pd.isna(piece) or len(piece)!=8:
        continue
      if all(hamming(piece, base) > 1 for base in thischunk):
            thischunk.append(piece)
    result = thischunk

    if len(pieces)>len(result):
      print("original:")
      print(pieces)
      print(len(pieces))
      print("result:")
      print(result)
      print(len(result))

      test_df.at[idx, 'UMIs_condensed'] = result
      test_df.at[idx, 'UMIs_condensed_count'] = len(result)
    else:
      test_df.at[idx, 'UMIs_condensed'] = pieces
      test_df.at[idx, 'UMIs_condensed_count'] = len(pieces)
  else:
    test_df.at[idx, 'UMIs_condensed'] = pieces
    test_df.at[idx, 'UMIs_condensed_count'] = len(pieces)

display(test_df)
test_df.to_csv('rapid_proc_%s_decodes_%iMM_UMIcondensed.csv' % (barcode, mm))

##Match encoding regions to BBs using a user-provided .csv files with BB assignments

In [ ]:
#Example code reflecting how encoding codes are matched to building blocks provided in a .csv by the user

df = pd.read_csv('rapid_proc_%s_decodes_%iMM_UMIcondensed.csv' % (barcode, mm))
# df = df.replace('unique', np.nan).dropna(axis=0)

df= df.replace("\(\'", '[', regex = True)
df= df.replace('\'\)', ']', regex = True)
df= df.replace("\'", '', regex = True)
print(df)
df['Code'] = df['Code'].apply(eval)
codes = pd.read_csv('coding scheme DEL19.csv', dtype = str)
#print(codes['code 1'])
new_df = pd.DataFrame(columns = ['BB code', 'sequence', 'UMI', 'UMIcount','Line#', 'ER1', 'ER2', 'ER3', 'ER4', 'ER5', 'ER6'])
c = 0
for count, entry in enumerate(df['Code'].values):
  # print(entry)
  #print(len(df.iloc[c, 1]))
  pos1 = entry[0]
  pos2 = entry[1]
  pos3 = entry[2]
  pos4 = entry[3]
  pos5 = entry[4]
  pos6 = entry[5]

  BSB = str(entry[0]) + "-" + str(entry[1])
  code1 =str(entry[2])+ "-" + str(entry[3])
  code2 =str(entry[4]) + "-" + str(entry[5])

  bb1 = codes[codes.eq(code1).any(axis=1)]['BB code'].values
  bb2 = codes[codes.eq(code2).any(axis=1)]['BB code'].values
  # (print(bb1,bb2))
  if bb1 != "" and bb2 != "": #only reads with full length matches will continue now.
    print(bb1,bb2)
    new_df.loc[len(new_df)] = [bb1, entry, df.iloc[c]['UMIs_condensed'], df.iloc[c]['UMIs_condensed_count'], df.iloc[c]['Linecount'],
                               entry[0],entry[1],entry[2],entry[3],entry[4],entry[5]]
    new_df.loc[len(new_df)] = [bb2, entry, df.iloc[c]['UMIs_condensed'], df.iloc[c]['UMIs_condensed_count'], df.iloc[c]['Linecount'],
                               entry[0],entry[1],entry[2],entry[3],entry[4],entry[5]]
    #print(new_df)
    c += 1
print(new_df)
new_df.to_csv("rapid_%s_BB_match.csv" % barcode)

##UMI trimming

In [ ]:
# Plot UMI's to observe distribution of counts. This should help inform the basis of the UMI cutoff.

# Plot as Log scale, Y axis

df= pd.read_csv("rapid_%s_BB_match.csv" % (barcode))
binwidth = 10
plt.hist(df['UMIcount'],
         bins = range(1, 1000 + binwidth, binwidth)
          ,log = True)
plt.title("UMI counts, " + barcode)
plt.xscale('log')

plt.show()

In [ ]:
# Plot as linear scale, Y axis


df= pd.read_csv("rapid_%s_BB_match.csv" % (barcode))
binwidth = 10
plt.hist(df['UMIcount'],
         bins = range(1, 1000 + binwidth, binwidth))
plt.title("UMI counts, " + barcode)
plt.xscale('log')

plt.show()

Trim files by UMI cutoff

In [ ]:
#set cutoff here
UMI_cutoff = 10

## code
df = pd.read_csv("rapid_%s_BB_match.csv" % (barcode))
df = df[df['UMIcount'] >= UMI_cutoff]
print(df)
df.to_csv("rapid_%s_BB_match_trimmed.csv" % (barcode))

In [ ]:
#take umi filtered beads and collapse on each BB
df = pd.read_csv("rapid_%s_BB_match_trimmed.csv" % (barcode))
df2 = df.groupby('BB code')[['sequence', 'ER1', 'ER2', 'ER3','ER4','ER5','ER6']].nunique()
#aggregate(lambda tdf: tdf.unique().tolist()).nunique
df2.to_csv("rapid_test_BB_match_%s_trimmed_bbcount.csv"% barcode)#"decode_"+barcode+"_2MM_bbcounts.csv")

In [ ]:
# Group by Encoding regions (ER)
df = pd.read_csv('rapid_%s_BB_match_trimmed.csv' % barcode)#("decode_"+barcode+"_2MM_trimmed.csv")
df2 = df.groupby('sequence').agg({"BB code": 'unique',
                                 "UMIcount": 'sum',
                                 'Line#':'nunique',
                                 'ER1':'unique',
                                 'ER2':'unique',
                                 'ER3':'unique',
                                 'ER4':'unique',
                                 'ER5':'unique',
                                 'ER6':'unique'})
df2.to_csv("rapid_%s_BB_match_byBead.csv"% barcode)#"decode_"+barcode+"_2MM_byBead_n.csv")
#df3.to_csv('CGATGT_decode_byBead_n2.csv')



In [ ]:
display(df2)

##Assess reads for recombination
  
  If sequences for beads appear with the same encoding regions despite a diversity of encoding regions used to prepare the library, throw out reads which are improbable due to sampling statistics and are more likely a product of recombination during sequencing.

In [ ]:
import ast
from ast import literal_eval
from math import log

from scipy.stats import poisson
# barcode = 'CGATGT'

In [ ]:
def hamming(s1, s2):
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))




df=pd.read_csv("rapid_%s_BB_match_byBead.csv"% barcode)
print(df)
# df['AggER']=df['ER3']+ "-"+ df['ER4'] + "-"+ df['ER5'] + "-"+ df['ER6']
df2 = df.groupby('BB code').agg({"sequence":', '.join,
                                 "UMIcount": 'sum',
                                 'Line#':'sum',
                                 'ER1':'nunique',
                                 'ER2':'nunique',
                                 'ER3':'nunique',
                                 'ER4':'nunique',
                                 'ER5':'nunique',
                                 'ER6':'nunique'})
# display(df2)
for i, row in df2.iterrows():
  # print("original")
  code = row['sequence']
  # print(code)
  if len(code) > 36:
    code = ast.literal_eval(code)
    uniquechunk=[]
    uniquepiece=[]
    samechunk = []
    samepiece = []
    for piece in code:
      chunk = piece[2:]
      # for base in uniquechunk:
      #   print(hamming(piece, base))
      if all(hamming(chunk, base) >= 1 for base in uniquechunk):
        uniquechunk.append(chunk)
        uniquepiece.append(piece) #if there is a diff in codes, pass
      else:
        count +=1
        samechunk.append(chunk)
        samepiece.append(piece)
      # poisson
    lam = len(code)/4
    k = len(samechunk) #reps
    p = poisson.pmf(k=k, mu=lam)
    count = 0
    if p < 0.05:
      print(samepiece)
      for line in samepiece:

        values = ', '.join(str(v) for v in line)
        values = "[" + values+ "]"
        # print(values)
        # print(values)
        df.drop(df.loc[df['sequence']==values].index, inplace=True)
        # df = df[df.sequence != values]



print(df)
df.to_csv('rapid_%s_BB_match_byBead_recomb.csv'% barcode)

## Sort hits by k-class or replicate class

In [ ]:
df = pd.read_csv('rapid_%s_BB_match_byBead_recomb.csv' % barcode)
df2 = df.sort_values(['UMIcount'], ascending=False)

df2 = df2.groupby(['BB code']).agg(Reps = ('sequence', 'nunique'),
                                   sumUMI = ('UMIcount', 'sum'),
                                   ER1   = ('ER1','nunique'),
                                   ER2   = ('ER2','nunique'),
                                   ER3   = ('ER3','nunique'),
                                   ER4   = ('ER4','nunique'),
                                   ER5   = ('ER5','nunique'),
                                   ER6   = ('ER6','nunique'))
# print(df2)
cols = ['ER1', 'ER2','ER3','ER4','ER5','ER6']
df2['nERs'] = df2[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
df2 = df2.sort_values(['Reps'], ascending=False)
# print(df2)
df2.to_csv("rapid_%s_BB_match_codesByBeadsorted_recomb.csv"% barcode)
display(df2)

#Aggregate data from negative (ortogonal control)screens

In [ ]:
# obtain all relevant files

dir = "/content/drive/MyDrive/Colab Notebooks"
files = glob.glob(dir+ "/*_codesByBeadsorted_recomb_trimmed.csv")
print(files)
# print(files[4])

In [ ]:
posneg = 4 #index of negative file
df1 = pd.read_csv(files[posneg])
print(df1)
df = pd.DataFrame(df1)
# df=df.drop(['ER1', 'ER2', 'ER3','ER4','ER5','ER6'], axis=1)
df = df.set_index(['BB code'])
# print(files)
columns_to_join = ['Reps', 'sumUMI', 'nERs']

for i, file in enumerate(files):
  if i ==posneg:
    continue
  base=os.path.basename(file)
  file_name = os.path.splitext(base)[0]
  # file_name = file_name.split(" ")[0]
  print(file_name)
  df_add = pd.read_csv(file)
  df_add = df_add.set_index(['BB code'])

  # print(df_add)
  df = df.join(df_add[columns_to_join], how='outer', rsuffix = str(" "+ file_name))
  # print(df)
df['in negative?'] = df['Reps']>=2 # replicate class cutoff for negative screen

#find all columns with replicates from the different barcodes and sum

reps_columns = df.filter(like='Reps ') ## will only take reps from positive screens
df['total_reps'] = reps_columns.sum(axis=1)
df['appearance, nscreens'] = reps_columns.count(axis=1)

#find all columns with sumUMIs from the different barcodes and sum

UMI_columns = df.filter(like='sumUMI ')
df['totalUMIs'] = UMI_columns.sum(axis=1)

#find all columns with unnamed indexes from the different barcodes and drop from df

drop_cols = df.filter(like='Unnamed')
df = df.drop(drop_cols,axis=1)
 #sort the DF to put all relevant info together
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv('agg_data.csv')
display(df)

# Model full products

##Code has been omitted from this portion to retain confidentiality of hit data##

Code will utilize aggregated data and based on SMILES, generate full products and truncates based upon the library build, retaining information regarding Unique UMI's per hit, appearance in *n* screens, and encoding region health.

#False discovery rate calculations
Calculate FDR for k class cutoff determination based on poisson sampling statistical probability

In [ ]:
import ast
from ast import literal_eval
from math import log

from scipy.stats import poisson
# barcode = 'CGATGT'

In [ ]:
# for n in range(5348):

lam = 479/3456
for k in range(8):
  # k = 0 #reps
  p = poisson.pmf(k=k, mu=lam)
  if k == 0:
    print(k," prob:" , p)#)
  if k > 0:
    print(k," prob:" , p)
